In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from pandas import DataFrame as df
import numpy as np
import csv
from itertools import combinations
from sklearn.model_selection import train_test_split
import math
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from scipy import stats
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA

In [2]:
otu = pd.read_csv("data/silva/freshwater/otu_filtered_freshwater.csv", sep = "\t", index_col=0)
print(otu.shape)
otu.head()

(18567, 5817)


,1039.P.Dois.Rios.SA,1039.L.Jacarepia.HB,1041.S008.2m.off.bottom,1041.M041.2m.off.bottom,1041.S008.10,1041.S008.150,1041.S008.200,1041.S019.100,1041.S019.20,1041.S019.5,...,945.P12.H4.lane5.NoIndex.L005,945.P2.G1.lane2.NoIndex.L002,945.P5.G4.lane2.NoIndex.L002,945.P14.E12.lane3.NoIndex.L003,945.P12.G6.lane5.NoIndex.L005,945.P6.H3.lane2.NoIndex.L002,945.P7.G4.lane2.NoIndex.L002,945.P6.B4.lane2.NoIndex.L002,945.P7.H10.lane2.NoIndex.L002,945.P7.G10.lane2.NoIndex.L002
#OTU ID,,,,,,,,,,,,,,,,,,,,,
FJ478961,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
JQ919464,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
JN023493,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GQ500796,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EF019723,1.0,415.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
qual_vecs = pd.read_csv("embeddings/silva/glove_emb_freshwater_500.txt", sep = " ", index_col = 0, header=None)
print(qual_vecs.shape)
qual_vecs.head()

(17001, 500)


,1,2,3,4,5,6,7,8,9,10,...,491,492,493,494,495,496,497,498,499,500
0,,,,,,,,,,,,,,,,,,,,,
EF111171,0.118641,-0.444386,0.246634,-0.072902,0.369961,-0.482379,0.209352,0.295594,-0.205730,-0.224248,...,-0.487010,-0.162772,-0.164275,0.077253,-0.066350,-0.093042,0.413554,0.312778,0.187127,0.264965
GU127198,-0.232628,-0.157525,0.062588,0.008834,-0.021595,0.167707,0.019661,0.065617,-0.344988,-0.036438,...,-0.033295,-0.600909,-0.023947,-0.106806,0.042348,-0.318011,-0.478798,0.254259,0.305406,0.338453
EU800861,0.295650,-0.652243,0.274248,0.516278,0.399282,0.126761,0.425216,-0.366479,-0.456424,-0.544973,...,-0.171479,0.091878,-0.288257,0.148256,0.017180,-0.111532,0.047036,0.363246,0.196964,0.169608
HM346776,0.356277,-0.179216,-0.235156,0.450923,0.303946,-0.117682,0.281581,-0.204243,-0.443172,0.490969,...,-0.347756,-0.378651,0.108651,-0.100085,0.146812,-0.232929,0.085037,-0.167525,0.137217,-0.043598
EU801674,0.013289,-0.416142,0.311181,-0.075872,0.284878,-0.066812,0.527253,-0.173661,-0.028396,-0.361644,...,-0.041885,-0.206095,-0.369141,-0.478056,-0.136812,-0.316054,0.315907,0.399233,0.462678,-0.153479


In [4]:
bools_drop = [i not in qual_vecs.index.values for i in otu.index.values]
drop = otu.index.values[bools_drop]
otu_drop = otu.drop(drop, axis = 0)
otu_drop.shape

(17000, 5817)

In [5]:
qual_vecs = qual_vecs.drop("<unk>", axis = 0)
qual_vecs_sort = qual_vecs.reindex(sorted(qual_vecs.index.values), axis = 0)
otu = otu_drop.reindex(sorted(otu_drop.index.values), axis = 0) #Organize otu rows to match taxa in qual_vecs

ntaxa = qual_vecs.shape[0]
print(ntaxa)
bools_correct = [qual_vecs_sort.index.values[i] == otu.index.values[i] for i in range(ntaxa)]

if (np.sum(bools_correct) == qual_vecs_sort.shape[0]) and (np.sum(bools_correct) == otu.shape[0]):
    print("Safe to continue")
else:
    print("STOP! Something is wrong.")

17000
Safe to continue


In [6]:
mapping = pd.read_csv("data/emp_qiime_mapping_release1.tsv.csv", sep = ",", index_col=0, encoding='latin1')
print("Mapping has shape: " + str(mapping.shape))

map_filt = mapping.loc[mapping['empo_3'] == "Water (non-saline)"]
print("After selecting for biome, mapping has shape " + str(map_filt.shape))

bools = [i in otu.columns.values for i in map_filt.index.values]
map_filt = map_filt.loc[bools]
print("After selecting just the samples present in the otu table: " + str(map_filt.shape))

otu_sort = otu.reindex(sorted(otu.columns.values), axis = 1)
map_filt_sort = map_filt.reindex(sorted(map_filt.index.values), axis = 0)
nsamples = map_filt.shape[0]
bools_correct = [map_filt_sort.index.values[i] == otu_sort.columns.values[i] for i in range(nsamples)]
print("After rearranging, we have " + str(np.sum(bools_correct)) + " matching samples")
if (np.sum(bools_correct) == map_filt_sort.shape[0]) and (np.sum(bools_correct) == otu_sort.shape[1]):
    print("Safe to continue")
else:
    print("STOP! Something is wrong.")

Mapping has shape: (27751, 75)
After selecting for biome, mapping has shape (5940, 75)
After selecting just the samples present in the otu table: (5817, 75)
After rearranging, we have 5817 matching samples
Safe to continue


In [7]:
#temperature
#phosphate
#ammonia
#map_filt_sort.loc[map_filt_sort['temperature_deg_c']]
bools =  ~map_filt_sort['temperature_deg_c'].isnull() 
map_temp = map_filt_sort.loc[~map_filt_sort['temperature_deg_c'].isnull()]
otu_temp = otu_sort.loc[:, bools]
bools_correct = [map_temp.index.values[i] == otu_temp.columns.values[i] for i in range(map_temp.shape[0])]
print("We will be working with " + str(np.sum(bools_correct)) + " samples that have temperature information")

We will be working with 4499 samples that have temperature information


In [8]:
#One final check after all transformations
qual_vecs = qual_vecs_sort
bools_correct = [qual_vecs.index.values[i] == otu_temp.index.values[i] for i in range(ntaxa)]
if (np.sum(bools_correct) == qual_vecs_sort.shape[0]) and (np.sum(bools_correct) == otu.shape[0]):
    print("Safe to continue")
else:
    print("STOP! Something is wrong.")

Safe to continue


In [11]:
otu_t = otu_temp.transpose()
print(otu_t.shape)
print(qual_vecs.shape)

(4499, 17000)
(17000, 500)


DEEP LEARNING PREDICTION

In [ ]:
#Declare architecture
#Input: taxa x property matrix 